In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json

with open('/Users/aassamidanov/Desktop/personal.json') as f:    
    dt = json.load(f)

In [2]:
dt

[{'header': {'version': '0.0.1',
   'incrementality': 'FULL_DATASET',
   'timestamp': '0'},
  'stations': [{'id': 'nsit',
    'status': {'state': 'CONNECTED', 'message': ''},
    'powerGenerationParameters': {'irradiation': 0.009999999776482582,
     'currentPowerOutput': 0.0,
     'powerGeneratedToday': 0.0,
     'windspeed': 0.0,
     'moduleTemperature': 0.0},
    'info': {'powerCapacity': 324.0,
     'name': 'IPGCL 1MWp NSIT',
     'location': {'latitude': 28.6091, 'longitude': 77.0351}},
    'inverterList': [{'id': 'NSIT_Inverter_1',
      'status': {'state': 'CONNECTED', 'message': ''},
      'powerGenerationParameters': {'currentPowerOutput': 0.0,
       'powerGeneratedToday': 0.0,
       'irradiation': 0.0,
       'windspeed': 0.0,
       'moduleTemperature': 0.0},
      'info': {'powerCapacity': 30.0, 'name': 'NSIT_Inverter_1'},
      'timestamp': '1578015904',
      'phaseComponents': [],
      'solarPanelGroup': []},
     {'id': 'NSIT_Inverter_2',
      'status': {'state': '

In [3]:
df_pre = json_normalize(json.loads(json_normalize(dt,'stations') .to_json(orient = "records"))).rename(index=str,
        columns={'id': 'solar_id', 'timestamp': 'solar_time'})

In [4]:
meta_info = [i for i in df_pre.drop(['inverterList', ], axis=1)]

In [5]:
df = json_normalize(json.loads(json_normalize(json.loads(df_pre.to_json(orient = "records")),
               record_path = ['inverterList'], meta=meta_info).to_json(orient = "records")))

In [6]:
df.head(20)

,id,info.location.latitude,info.location.longitude,info.name,info.powerCapacity,phaseComponents,powerGenerationParameters.currentPowerOutput,powerGenerationParameters.irradiation,powerGenerationParameters.moduleTemperature,powerGenerationParameters.powerGeneratedToday,powerGenerationParameters.windspeed,solarPanelGroup,solar_id,solar_time,status.message,status.state,timestamp
0,NSIT_Inverter_1,28.6091,77.0351,NSIT_Inverter_1,30.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015904
1,NSIT_Inverter_2,28.6091,77.0351,NSIT_Inverter_2,30.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015901
2,NSIT_Inverter_3,28.6091,77.0351,NSIT_Inverter_3,50.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015901
3,NSIT_Inverter_4,28.6091,77.0351,NSIT_Inverter_4,30.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015903
4,NSIT_Inverter_5,28.6091,77.0351,NSIT_Inverter_5,30.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015903
5,NSIT_Inverter_6,28.6091,77.0351,NSIT_Inverter_6,50.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015906
6,NSIT_Inverter_7,28.6091,77.0351,NSIT_Inverter_7,50.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015905
7,NSIT_Inverter_8,28.6091,77.0351,NSIT_Inverter_8,50.00,[],0.000,0.0,0.0,0.00,0.0,[],nsit,1578015768,,CONNECTED,1578015901
8,DELHIHAAT_inverter1,28.6700,77.1800,DELHIHAAT_inverter1,50.00,[],0.210,0.0,0.0,0.01,0.0,[],delhihaat,1578016983,,CONNECTED,1578016804
9,AUROBINDO_INV1,28.6700,77.1800,AUROBINDO_INV1,50.00,[],0.315,0.0,0.0,0.01,0.0,[],aurobindo,1578016976,,CONNECTED,1578016800
